<a href="https://colab.research.google.com/github/shaheerzubery/Whisper/blob/main/transcripts_with_speaker_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Our Today lecture is about How to get the diarization and timestamp using Whisper and pynnote


below are some requirement that you need to do

Notes on usage:

- Make sure to [change runtime to GPU](https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm).
- The transcript will be saved in Files, which you can find in the menu on the left.
- Change the number of speakers below if different from two.
- Pick a bigger model if you want more accuracy and a smaller model if you want the program to run faster ([more info](https://github.com/openai/whisper#available-models-and-languages)).
- If you know the language being spoken is English, then change language to 'English' as this improves performance.


High level overview of what's happening here:


1.   I'm using Open AI's Whisper model to seperate audio into segments and generate transcripts.
2.   I'm then generating speaker embeddings for each segments.
3.   Then I'm using agglomerative clustering on the embeddings to identify the speaker for each segment.

Let me know if I can make it better!


In [ ]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving audio.mp3 to audio.mp3


now select the speaker language and model

In [ ]:
num_speakers = 2

language = 'English'

model_size = 'large'


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


This is just to convert to encode to UTF 8

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

Importing the required dependencies

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

This is a check whether an audio is wav or not

In [ ]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

loading the model

In [ ]:
model = whisper.load_model(model_size)

100%|██████████████████████████████████████| 2.87G/2.87G [00:23<00:00, 134MiB/s]


generate the segment of it

In [ ]:
result = model.transcribe(path)
segments = result["segments"]

This helps to get the frames rate and duration

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

This is pyannote built in function

In [ ]:
audio = Audio()

In [ ]:
# def segment_embedding(segment):
#     start = segment["start"]
#     # Whisper overshoots the end timestamp in the last segment
#     end = min(duration, segment["end"])
#     clip = Segment(start, end)
#     waveform, sample_rate = audio.crop(path, clip)

#     # Reshape the waveform tensor to match the expected input shape
#     waveform = waveform.unsqueeze(0)

#     return embedding_model(waveform)


THis helps to split the audios into wavform

In [ ]:
def segment_embedding(segment):
    start = segment["start"]
    # Whisper overshoots the end timestamp in the last segment
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(path, clip)

    # Convert waveform to single channel
    waveform = waveform.mean(dim=0, keepdim=True)

    return embedding_model(waveform.unsqueeze(0))


Here we create the embeddings

In [ ]:

embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)


embeddings = np.nan_to_num(embeddings)

Here we are using agglomerative clustering to cluster same channels as 1 speaker

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w",encoding="UTF-8")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [ ]:
print(open('transcript.txt','r',encoding="UTF-8").read())


SPEAKER 1 0:00:00
Thank you for choosing Mount Marnie Geo Service Center. This is Mallory speaking and I can 
SPEAKER 2 0:00:07
help you. Oh, hello. I made a protocol for us to book an appointment for my first service 
SPEAKER 1 0:00:16
of a bike car. Okay, yeah, we can get that scheduled for you. And have you had a service 
SPEAKER 2 0:00:23
here before? No. Okay. And you said this is the first service? That is the first service of my car. Okay. And what kind of vehicle is it? Say again? What kind of vehicle is it? The Ford Camry 1020. Okay. Yeah, we can set that appointment up for you and I'll just 
SPEAKER 1 0:00:49
make you a new account. Did you purchase it here by chance? Say again? Did you purchase 
SPEAKER 2 0:00:56
it here with us at Mount Martin? No. No? Okay. All right. And do you have an account? 
SPEAKER 1 0:01:04
Like, I have your phone number under here and it's coming up under Mr. Wynn. Is that 
SPEAKER 2 0:01:08
you? Yeah, that is my name. That is Mr. Wynn. Okay. Okay

This is how you can get the speaker dirization